<a href="https://colab.research.google.com/github/matteogianferrari/homelens-ca/blob/main/model/notebooks/homelens_ca_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HomeLens CA DL Model Research**

# **Setup**


---
**OS Environment Variables**

In [1]:
from google.colab import userdata
import os

# Sets environ variables for GitHub
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')
os.environ['USER'] = userdata.get('USER')

# Sets environ variables for MLFlow
os.environ['MLFLOW_TRACKING_USERNAME'] = userdata.get('USER')
os.environ['MLFLOW_TRACKING_PASSWORD'] = userdata.get('MLFLOW_TRACKING_PASSWORD')

---
**Libraries Install**

In [2]:
!pip install dagshub mlflow pynvml --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.2/258.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.5/637.5 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

---
**Libraries imports**

In [3]:
import tensorflow as tf

import mlflow
from mlflow.tracking import MlflowClient

import dagshub

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

---
**Dataset import**

In [4]:
# Clones the repo and changes dir
!git clone https://${GITHUB_TOKEN}@github.com/${USER}/homelens-ca.git
%cd homelens-ca/

Cloning into 'homelens-ca'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 34 (delta 4), reused 31 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 1.93 MiB | 25.96 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/homelens-ca


In [5]:
# Changes dir
%cd model/data/dataset

# Imports the dataset and the labels
X = pd.read_csv('X.csv').to_numpy()
y = pd.read_csv('y.csv').to_numpy()

# Applies transformation to labels
y = np.log1p(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

/content/homelens-ca/model/data/dataset


---
**DagsHub and MLFlow**

In [11]:
# Init DagsHub
dagshub.init(repo_owner='matteogianferrari', repo_name='homelens-ca', mlflow=True)
TRACKING_URI = "https://dagshub.com/matteogianferrari/homelens-ca.mlflow"

# Sets MLFlow tracking URI
mlflow.set_tracking_uri(TRACKING_URI)

# Sets MLFLow experiment name
experiment_name = "HomeLens CA-DL Research"

Initialized MLflow to track repo "matteogianferrari/homelens-ca"

Repository matteogianferrari/homelens-ca initialized!

# **Research and Training DL Models**

---
**Model Hyperparameters**

Defines the Hyperparameters for the model.

In [12]:
epochs = 400
initial_learning_rate = 1e-3
minimum_learning_rate = 1e-8
patience_es=50
patience_lr=15
decaying_factor=0.5
batch_size=128
validation_slip=0.1
loss_func='mse'

---
**Training and Testing**

In [13]:
import pynvml

# Sets the MLFlow experiment
mlflow.set_experiment(experiment_name)

# Autolog
mlflow.keras.autolog()



# Defines the model
def build_dynamic_model(input_shape, n_repeats):
    """
    Builds a neural network model using TensorFlow's Functional API with repeated blocks.

    Args:
        input_shape (tuple): Shape of the input data.
        n_repeats (int): Number of times to repeat the block of layers.

    Returns:
        tf.keras.Model: A compiled Keras model.
    """
    # Input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = inputs

    # Define the repeated block
    for _ in range(n_repeats):
        x = tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.2)(x)

        x = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.2)(x)

        x = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Output layer
    outputs = tf.keras.layers.Dense(1)(x)

    # Create the model
    model = tf.keras.Model(inputs, outputs)
    return model

# Example usage
n_repeats = 3
input_shape = (15,)
model = build_dynamic_model(input_shape, n_repeats)



# Set up a decaying learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

# You can now compile your model using this optimizer
model.compile(
    optimizer=optimizer,
    loss=loss_func,
    metrics=['r2_score']
)

# Creates an Early-Stopping callback
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_r2_score',
    mode='max',
    patience=patience_es,
    restore_best_weights=True
)

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=decaying_factor,
    patience=patience_lr,
    min_lr=minimum_learning_rate
)


# Train the model and log with MLFlow
with mlflow.start_run(log_system_metrics=True):
    mlflow.log_param("patience", patience_es)
    mlflow.log_param("loss_func", loss_func)

    history = model.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=validation_slip,
        callbacks=[es_callback, lr_callback]
    )

    # Test the model and log with MLFlow
    test_loss, test_r2_score = model.evaluate(X_test, y_test)
    mlflow.log_metric("test_loss", test_loss)
    mlflow.log_metric("test_r2_score", test_r2_score)

2025/01/24 19:35:28 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 136.9767 - r2_score: -487.5059

116/116 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - loss: 136.8743 - r2_score: -487.1750 - val_loss: 98.4595 - val_r2_score: -366.2347 - learning_rate: 0.0010
Epoch 2/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 74.0857 - r2_score: -267.2707 - val_loss: 24.0057 - val_r2_score: -88.5364 - learning_rate: 0.0010
Epoch 3/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 13.2012 - r2_score: -46.5607 - val_loss: 1.4126 - val_r2_score: -4.2689 - learning_rate: 0.0010
Epoch 4/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5920 - r2_score: -1.1303 - val_loss: 0.1432 - val_r2_score: 0.4659 - learning_rate: 0.0010
Epoch 5/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1723 - r2_score: 0.3773 - val_loss: 0.0882 - val_r2_score: 0.6709 - learning_rate: 0.0010
Epoch 6/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1528 - r2_score: 0.4532 - val_loss: 0.0857 - val_r2_score: 0.6805 - learning_rate: 0.0010
Epoch 7/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1389 - r

116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1259 - r2_score: 0.5459 - val_loss: 0.0908 - val_r2_score: 0.6612 - learning_rate: 0.0010
Epoch 9/400
115/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1193 - r2_score: 0.5714

116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1193 - r2_score: 0.5714 - val_loss: 0.0765 - val_r2_score: 0.7147 - learning_rate: 0.0010
Epoch 10/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1159 - r2_score: 0.5853 - val_loss: 0.0767 - val_r2_score: 0.7141 - learning_rate: 0.0010
Epoch 11/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1105 - r2_score: 0.5994 - val_loss: 0.0709 - val_r2_score: 0.7357 - learning_rate: 0.0010
Epoch 12/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1087 - r2_score: 0.6162 - val_loss: 0.0696 - val_r2_score: 0.7404 - learning_rate: 0.0010
Epoch 13/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1052 - r2_score: 0.6188 - val_loss: 0.0669 - val_r2_score: 0.7506 - learning_rate: 0.0010
Epoch 14/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1002 - r2_score: 0.6371 - val_loss: 0.0716 - val_r2_score: 0.7328 - learning_rate: 0.0010
Epoch 15/400
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1000 - r2_score: 0.6427 -

2025/01/24 19:40:09 WARNING mlflow.system_metrics.system_metrics_monitor: Failed to get mlflow run: API request to https://dagshub.com/matteogianferrari/homelens-ca.mlflow/api/2.0/mlflow/runs/get failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /matteogianferrari/homelens-ca.mlflow/api/2.0/mlflow/runs/get?run_uuid=3b46673807a44bd18cdfc89d0f4b6b72&run_id=3b46673807a44bd18cdfc89d0f4b6b72 (Caused by ResponseError('too many 429 error responses')).


🏃 View run mysterious-stork-964 at: https://dagshub.com/matteogianferrari/homelens-ca.mlflow/#/experiments/0/runs/3b46673807a44bd18cdfc89d0f4b6b72
🧪 View experiment at: https://dagshub.com/matteogianferrari/homelens-ca.mlflow/#/experiments/0


2025/01/24 19:43:05 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/01/24 19:43:05 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
